# Reproduce DDGemb method

In [1]:
import pandas as pd
import numpy as np


In [2]:
path = 'data/'
df_fireprot = pd.read_csv(path + 'fireprotdb_results.csv')


C:\Users\badea\AppData\Local\Temp\ipykernel_14028\2789855006.py:2: DtypeWarning: Columns (23,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fireprot = pd.read_csv(path + 'fireprotdb_results.csv')


In [4]:
df_fireprot.head()

,experiment_id,protein_name,uniprot_id,pdb_id,chain,position,wild_type,mutation,ddG,dTm,...,technique,technique_details,pH,tm,notes,publication_doi,publication_pubmed,hsw_job_id,datasets,sequence
0,LL000001,Haloalkane dehalogenase,P59336,1CQW,A,245,V,L,NaN,2.1,...,NaN,NaN,NaN,52.5,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
1,LL000002,Haloalkane dehalogenase,P59336,1CQW,A,95,L,V,NaN,-0.4,...,NaN,NaN,NaN,50.0,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
2,LL000004,Haloalkane dehalogenase,P59336,1CQW,A,176,C,F,NaN,5.2,...,NaN,NaN,NaN,55.6,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
3,LL000005,Haloalkane dehalogenase,P59336,1CQW,A,171,G,Q,NaN,3.1,...,NaN,NaN,NaN,53.5,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...
4,LL000006,Haloalkane dehalogenase,P59336,1CQW,A,148,T,L,NaN,1.1,...,NaN,NaN,NaN,51.5,NaN,NaN,NaN,xfyu58,NaN,MSEIGTGFPFDPHYVEVLGERMHYVDVGPRDGTPVLFLHGNPTSSY...


In [5]:
print(df_fireprot.columns)

Index(['experiment_id', 'protein_name', 'uniprot_id', 'pdb_id', 'chain',
       'position', 'wild_type', 'mutation', 'ddG', 'dTm', 'is_curated', 'type',
       'derived_type', 'interpro_families', 'conservation', 'is_essential',
       'correlated_positions', 'is_back_to_consensus', 'secondary_structure',
       'asa', 'is_in_catalytic_pocket', 'is_in_tunnel_bottleneck', 'b_factor',
       'method', 'method_details', 'technique', 'technique_details', 'pH',
       'tm', 'notes', 'publication_doi', 'publication_pubmed', 'hsw_job_id',
       'datasets', 'sequence'],
      dtype='object')


In [7]:
protein_counts_1 = df_fireprot["protein_name"].value_counts().reset_index()
protein_counts_1

,protein_name,count
0,Subtilisin-chymotrypsin inhibitor-2A,12276
1,Tyrosine-protein kinase Fyn,2488
2,Halohydrin dehalogenase,2480
3,ADHA,2404
4,Immunoglobulin G-binding protein G,2220
...,...,...
205,"Ferredoxin, heterocyst",1
206,Non-specific lipid-transfer protein,1
207,S-adenosylmethionine synthase isoform type-1,1
208,Tetracycline repressor protein class D,1


In [14]:
columns_to_keep = ['experiment_id', 'protein_name', 'uniprot_id', 'pdb_id', 'chain',
       'position', 'wild_type', 'mutation', 'ddG', 'sequence']
df_fireprot = df_fireprot[-df_fireprot['ddG'].isna()]
df_subset = df_fireprot[columns_to_keep]

In [18]:
df_subset[df_subset["protein_name"].isna()].pdb_id.unique()

array(['2IMM', '1YYX'], dtype=object)

In [16]:
print("The dataset contains {} rows".format(len(df_subset)))

The dataset contains 39177 rows
